In [1]:
import os
import sys
import django
from django.db import connection
# from tqdm import tqdm
import logging
import pandas as pd
from datetime import datetime

sys.path.insert(0, "/Users/victornguyen/Sites/07.book_management/")

In [2]:
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "book_management.settings")
django.setup()
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)
logger = logging.getLogger('User similarity calculator')

# Import Models
from title.models import Book
from transaction.models import Master, Detail
from main_site.models import Rating

In [3]:
def cal_point(sample):
    hire_date_length = abs(sample.return_date - sample.hire_date)
    due_date_length = abs(sample.due_date - sample.hire_date)
    ratio = hire_date_length / due_date_length
    sample['hire_date_length'] = hire_date_length
    sample['due_date_length'] = due_date_length
    sample['ratio_hire_date'] = round(ratio, 3)
    if (ratio > 2).bool():
        sample['point'] = 4.50
    elif (ratio > 1).bool():
        ratio_overdue = abs(1 - ratio)
        sample['point'] = 3 + (7 - (7 * ratio_overdue))
    else:
        sample['test'] = abs((1 - ratio) / (ratio - 1))
        sample['point'] = 3 + (7 * ratio)
    sample['point'] = round(sample['point'], 2)
    return sample

In [4]:
book_df = pd.DataFrame(list(Book.objects.all().values()))
# Change transaction_master_db to data frame
transaction_master_df = pd.DataFrame(list(Master.objects.all().values()))
# Change name column
transaction_master_df.rename(columns={'date': 'hire_date'},
                             inplace=True)
# Change transaction_detail_db to data frame
transaction_detail_df = pd.DataFrame(list(Detail.objects.all().values()))
# Join two data frame to one
transaction_df = pd.merge(transaction_master_df, transaction_detail_df, left_on='id', right_on='transaction_id')
transaction_df = pd.merge(transaction_df, book_df[['barcode', 'title_id']], left_on="book_id",
                          right_on="barcode",
                          how="left")
transaction_df = transaction_df.drop(['id_x', 'transaction_id'], axis=1)
transaction_df.describe(include = "all")

,hire_date,user_id,book_id,due_date,hire_type,id_y,price,return_date,status,barcode,title_id
count,187,187.000000,187,187,187.000000,187.000000,187.000000,173,187.000000,187,187.000000
unique,96,NaN,122,66,NaN,NaN,NaN,173,NaN,122,NaN
top,2017-09-14 00:00:00,NaN,52076-130,2018-02-15 00:00:00,NaN,NaN,NaN,2018-01-15 12:13:15,NaN,52076-130,NaN
freq,11,NaN,5,41,NaN,NaN,NaN,1,NaN,5,NaN
first,2017-02-15 00:00:00,NaN,NaN,2017-06-13 00:00:00,NaN,NaN,NaN,2017-06-12 16:01:18,NaN,NaN,NaN
last,2019-03-27 00:00:00,NaN,NaN,2019-07-15 00:00:00,NaN,NaN,NaN,2019-04-18 09:15:45,NaN,NaN,NaN
mean,NaN,62.556150,NaN,NaN,1.556150,94.000000,45054.759358,NaN,1.171123,NaN,76.967914
std,NaN,33.796685,NaN,NaN,0.539616,54.126395,30322.839972,NaN,0.560933,NaN,44.977163
min,NaN,8.000000,NaN,NaN,1.000000,1.000000,0.000000,NaN,1.000000,NaN,1.000000
25%,NaN,36.500000,NaN,NaN,1.000000,47.500000,17776.000000,NaN,1.000000,NaN,34.000000


In [5]:
transaction_df[transaction_df.return_date < transaction_df.hire_date]

,hire_date,user_id,book_id,due_date,hire_type,id_y,price,return_date,status,barcode,title_id
101,2018-01-31,77,53357-80,2018-07-15,2,102,100000.00,2018-01-30 11:17:37,1,53357-80,31
111,2018-04-04,11,021023078,2018-07-15,2,112,100000.00,2018-03-28 15:29:50,1,021023078,164
114,2018-04-06,29,58584-70,2018-04-13,1,115,18281.00,2018-04-04 14:36:26,1,58584-70,88
117,2018-05-09,88,53692-60,2018-05-16,1,118,20553.50,2018-04-10 10:05:24,1,53692-60,28
122,2018-06-08,28,58627-100,2018-06-15,1,123,17776.00,2018-06-04 08:11:51,1,58627-100,54
126,2018-06-12,85,53685-80,2018-06-19,1,127,9898.00,2018-06-11 15:25:40,1,53685-80,104
127,2018-06-19,44,53488-90,2018-08-31,2,128,67569.00,2018-06-12 12:09:59,1,53488-90,98
129,2018-06-29,23,53359-70,2018-07-06,1,130,15453.00,2018-06-19 14:57:48,1,53359-70,30
131,2018-07-05,76,021022507,2018-07-12,1,132,25000.00,2018-06-29 13:35:26,1,021022507,152
132,2018-07-14,32,58588-60,2018-07-21,1,133,20048.50,2018-07-11 12:08:28,1,58588-60,66


In [6]:
transaction_df[pd.isna(transaction_df.return_date)]

,hire_date,user_id,book_id,due_date,hire_type,id_y,price,return_date,status,barcode,title_id
21,2017-09-14,43,53398-60,2018-02-15,2,22,100000.00,NaT,3,53398-60,97
52,2017-09-22,25,53664-90,2018-02-15,2,53,64337.00,NaT,3,53664-90,14
62,2017-09-26,54,53400-100,2018-02-15,2,63,59489.00,NaT,3,53400-100,130
158,2019-01-03,91,021022407,2019-01-10,1,159,25000.00,NaT,1,021022407,157
160,2019-01-14,46,58583-70,2019-01-21,1,161,16538.75,NaT,1,58583-70,100
171,2019-02-12,115,53663-70,2019-07-15,2,172,57368.00,NaT,3,53663-70,42
172,2019-02-12,113,58578-60,2019-02-26,3,173,0.00,NaT,1,58578-60,20
173,2019-02-13,69,53499-60,2019-07-15,2,174,55550.00,NaT,1,53499-60,133
178,2019-02-28,35,53613-80,2019-07-15,2,179,52217.00,NaT,3,53613-80,106
179,2019-02-28,62,52076-130,2019-07-15,2,180,66256.00,NaT,1,52076-130,91


In [7]:
transaction_df['return_date'] = transaction_df['return_date'].fillna(datetime.now())

In [8]:
transaction_df.describe(include = 'all')

,hire_date,user_id,book_id,due_date,hire_type,id_y,price,return_date,status,barcode,title_id
count,187,187.000000,187,187,187.000000,187.000000,187.000000,187,187.000000,187,187.000000
unique,96,NaN,122,66,NaN,NaN,NaN,174,NaN,122,NaN
top,2017-09-14 00:00:00,NaN,52076-130,2018-02-15 00:00:00,NaN,NaN,NaN,2019-06-29 02:30:41.322024,NaN,52076-130,NaN
freq,11,NaN,5,41,NaN,NaN,NaN,14,NaN,5,NaN
first,2017-02-15 00:00:00,NaN,NaN,2017-06-13 00:00:00,NaN,NaN,NaN,2017-06-12 16:01:18,NaN,NaN,NaN
last,2019-03-27 00:00:00,NaN,NaN,2019-07-15 00:00:00,NaN,NaN,NaN,2019-06-29 02:30:41.322024,NaN,NaN,NaN
mean,NaN,62.556150,NaN,NaN,1.556150,94.000000,45054.759358,NaN,1.171123,NaN,76.967914
std,NaN,33.796685,NaN,NaN,0.539616,54.126395,30322.839972,NaN,0.560933,NaN,44.977163
min,NaN,8.000000,NaN,NaN,1.000000,1.000000,0.000000,NaN,1.000000,NaN,1.000000
25%,NaN,36.500000,NaN,NaN,1.000000,47.500000,17776.000000,NaN,1.000000,NaN,34.000000


In [9]:
transaction_df.head()

,hire_date,user_id,book_id,due_date,hire_type,id_y,price,return_date,status,barcode,title_id
0,2017-02-15,37,53683-70,2017-07-15,2,1,59792.0,2018-03-19 11:25:23,1,53683-70,23
1,2017-02-15,39,53398-70,2017-07-15,2,2,100000.0,2017-07-17 10:08:27,1,53398-70,97
2,2017-06-06,111,53683-60,2017-06-13,1,3,14948.0,2017-06-12 16:01:18,1,53683-60,23
3,2017-07-01,64,58584-80,2017-08-31,2,4,73124.0,2018-03-19 11:17:35,1,58584-80,88
4,2017-06-19,25,53664-90,2017-08-31,2,5,64337.0,2017-09-22 14:23:02,1,53664-90,14


In [10]:
transaction_df = transaction_df.groupby('id_y').apply(cal_point).reset_index(drop=True)
transaction_df[transaction_df.return_date < transaction_df.hire_date]

,hire_date,user_id,book_id,due_date,hire_type,id_y,price,return_date,status,barcode,title_id,test,hire_date_length,due_date_length,ratio_hire_date,point
101,2018-01-31,77,53357-80,2018-07-15,2,102,100000.00,2018-01-30 11:17:37,1,53357-80,31,1,0 days 12:42:23,165 days,0.003,3.02
111,2018-04-04,11,021023078,2018-07-15,2,112,100000.00,2018-03-28 15:29:50,1,021023078,164,1,6 days 08:30:10,102 days,0.062,3.44
114,2018-04-06,29,58584-70,2018-04-13,1,115,18281.00,2018-04-04 14:36:26,1,58584-70,88,1,1 days 09:23:34,7 days,0.199,4.39
117,2018-05-09,88,53692-60,2018-05-16,1,118,20553.50,2018-04-10 10:05:24,1,53692-60,28,28,28 days 13:54:36,7 days,4.083,4.50
122,2018-06-08,28,58627-100,2018-06-15,1,123,17776.00,2018-06-04 08:11:51,1,58627-100,54,1,3 days 15:48:09,7 days,0.523,6.66
126,2018-06-12,85,53685-80,2018-06-19,1,127,9898.00,2018-06-11 15:25:40,1,53685-80,104,1,0 days 08:34:20,7 days,0.051,3.36
127,2018-06-19,44,53488-90,2018-08-31,2,128,67569.00,2018-06-12 12:09:59,1,53488-90,98,1,6 days 11:50:01,73 days,0.089,3.62
129,2018-06-29,23,53359-70,2018-07-06,1,130,15453.00,2018-06-19 14:57:48,1,53359-70,30,9,9 days 09:02:12,7 days,1.340,7.62
131,2018-07-05,76,021022507,2018-07-12,1,132,25000.00,2018-06-29 13:35:26,1,021022507,152,1,5 days 10:24:34,7 days,0.776,8.43
132,2018-07-14,32,58588-60,2018-07-21,1,133,20048.50,2018-07-11 12:08:28,1,58588-60,66,1,2 days 11:51:32,7 days,0.356,5.49


In [ ]:

# New data frame with user_id, title_id, point
user_rating = pd.DataFrame()
user_rating['user_id'] = transaction_df.user_id
user_rating['title_id'] = transaction_df.title_id
user_rating['point'] = transaction_df.point
user_rating_df = user_rating.groupby(['user_id', 'title_id']).mean().round(3).reset_index()
user_rating_df.head()